In [3]:
import functions as f
from functions import load_songs_from_xml, random_sample_df

In [60]:
import importlib

In [6]:
file = "BREAK_ALL.nml"
file = "BreakBeat not listened.nml"

df = load_songs_from_xml(file)

In [8]:
df

,Artist,Title,File_Path,Play_Count,Comment,BPM,Musical_Key_Int,Color,Musical_Key
0,Native Instruments,Bell AtSteppa,//ProgramData//Native Instruments//Traktor Pro...,,None,133.031677,,,
1,Badboe & Break the Box,Test the Bass (Original Mix),//DJ//BREAKBEAT//01-badboe_and_break_the_box_-...,2,save the scene..,134.996994,8,,G#
2,"Deekline, Nubreed, Phil K, Ransom",Bushpig,//DJ//BREAKBEAT//Bushpig.mp3,2,wrong rip begin,136.001755,7,,G
3,Christian J,Understand (Original Mix),//DJ//BREAKBEAT//Christian J - Understand(1).mp3,,tempooooo,130.721283,21,7,Am
4,Flack.su,T-Break (Version 2),//DJ//BREAKBEAT//Flack.su - T-Break (Version 2...,1,simple break,134.000061,18,2,F#m
...,...,...,...,...,...,...,...,...,...
1563,None,VA - Sound Of Acid Core Vol. 5 (1998) - Brain ...,//DJ//VA - Sound Of Acid Core Vol. 5 (1998) - ...,1,None,141.849503,15,,D#m
1564,None,c_major,//Users//Gergo Gyori//Desktop//c_major.wav,1,None,135.999619,0,,C
1565,None,c_minor,//Users//Gergo Gyori//Desktop//c_minor.wav,1,None,135.999619,12,,Cm
1566,None,NAPT - Wah,//DOWNLOAD//NAPT - Wah.mp3,2,None,130.001022,16,,Em


In [46]:
df[df['Musical_Key'] == ""]

,Artist,Title,File_Path,Play_Count,Comment,BPM,Musical_Key_Int,Color,Musical_Key
0,Native Instruments,Bell AtSteppa,//ProgramData//Native Instruments//Traktor Pro...,0,None,133.031677,None,0,


In [9]:
import pandas as pd

def random_sample_df(df, n_samples, random_state=None):
  """
  Randomly samples a specified number of rows from a pandas DataFrame.

  Args:
    df: The pandas DataFrame to sample from.
    n_samples: The number of samples to draw.
    random_state: An optional seed for the random number generator.

  Returns:
    A new pandas DataFrame containing the random sample.

  Raises:
    ValueError: If the requested number of samples is greater than the
                number of rows in the DataFrame.
  """

  # Check if the requested number of samples is valid
  if n_samples > len(df):
    raise ValueError("Number of samples cannot be greater than the number of rows in the dataframe.")

  # Sample the dataframe using random sampling
  return df.sample(n=n_samples, random_state=random_state)


# Sample 3 rows randomly
sampled_df = random_sample_df(df, 100)

print(sampled_df)

                    Artist                                              Title  \
436               Sabotage                  I Saw the Future (Sabotage Remix)   
34             Vigi & Flip                                    Freak Frequency   
1333         Adam Freeland                                               HATE   
232             Meat Katie                                Yeah - Jamie McHugh   
1288                  None  Sergey Hairulin group - Damaged (Steve Maccabe...   
...                    ...                                                ...   
888    Deekline And Wizard                   Angels (Ft Yolanda) (Napt Remix)   
196             Meat Katie                      All I Need (Lee Coombs Remix)   
1096       Blatta & Inesha                                   Enter the Dragon   
1373                  None                                          Bassism 2   
204   Meat Katie & Aquasky                      Feathers (Marco Bailey Remix)   

                           

In [10]:
sampled_df

,Artist,Title,File_Path,Play_Count,Comment,BPM,Musical_Key_Int,Color,Musical_Key
436,Sabotage,I Saw the Future (Sabotage Remix),//DJ//ALBUMOK//VA-Coastal Breaks II-2 CD[320]/...,,None,126.824951,23,,Bm
34,Vigi & Flip,Freak Frequency,//DJ//ALBUMOK//Adam Freeland - Tectonics 2000/...,,None,130.845657,14,,Dm
1333,Adam Freeland,HATE,//DJ//SPOTIFY//Rádióban tetszett//HATE.mp3,,None,134.997284,14,,Dm
232,Meat Katie,Yeah - Jamie McHugh,//DJ//ALBUMOK//Label - LOT-49//LOT49CD 002_-_L...,,None,128.993546,2,,D
1288,None,Sergey Hairulin group - Damaged (Steve Maccabe...,//DJ//SELECTOR//Sergey Hairulin group - Damage...,,None,128.000305,12,,Cm
...,...,...,...,...,...,...,...,...,...
888,Deekline And Wizard,Angels (Ft Yolanda) (Napt Remix),//DJ//ALBUMOK//[torrents.ru]_breaks_pack_march...,,8=D~,132.000534,14,,Dm
196,Meat Katie,All I Need (Lee Coombs Remix),//DJ//ALBUMOK//Label - LOT-49//LOT49 005_-_Mea...,,None,131.676849,5,,F
1096,Blatta & Inesha,Enter the Dragon,//DJ//BREAKBEAT-FOLDERS//JELLEMTELEN//Blatta_I...,,Uhtempóóó7 Huh!,135.000000,12,7,Cm
1373,None,Bassism 2,//DJ//SPOTIFY//YOUTUBE//Bassism 2.mp3,,None,133.119949,17,,Fm


In [40]:
df[df.Play_Count == 1]

,Artist,Title,File_Path,Play_Count,Comment
0,Shimon & Nixon,Around The Edge,//DJ//BREAKBEAT//Around The Edge.mp3,1,None
1,Drum Monkeys,Batty Batucada (Plaza De Funk Remix) - Mixed V...,//DJ//BREAKBEAT//Batty Batucada (Plaza De Funk...,1,diszkós
6,Evil Nine,For Lovers Not Fighters,//DJ//BREAKBEAT//For Lovers Not Fighters.mp3,1,None
7,"Wavewhore, Bex Riley",Funk Pill,//DJ//BREAKBEAT//Funk Pill.mp3,1,None
8,"Stanton Warriors, Ruby Goe, Hollywood Holt",Get Up - Original Mix,//DJ//BREAKBEAT//Get Up - Original Mix.mp3,1,None
...,...,...,...,...,...
734,The Freestylers,Hold Up Your Hands,"//DJ//The Freestylers//Freestylers, the - We R...",1,lazulós
735,The Freestylers,Warning,"//DJ//The Freestylers//Freestylers, the - We R...",1,ragás
742,The Prodigy,Run With The Wolves,//DJ//The Prodigy - Invaders Must Die (2009)//...,1,None
749,Robosapiens,FSociety,//DJ//BREAKBEAT//FSociety.mp3,1,None


In [18]:
set(df.Play_Count)

{None}